# <h1><center> Classificateur de Modèles de voitures</center></h1>
 <img src="logo/image.jpeg">

# Import des Modules

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import torchvision.models as models
from PIL import Image
import json
from matplotlib.ticker import FormatStrFormatter
import os

# Module de gestion du GPU
!pip install GPUtil
from numba import cuda
from GPUtil import showUtilization as gpu_usage


# **ETAPES** <p>
Etape 0: Libération du cache de la carte graphique <p>
Etape 1: Chargement des Datasets <p>
Etape 2: Transformation des Datasets<p>
Etape 3: Creation du Modele <p>
Etape 4: Entrainement du Modele<p>
Etape 5: Sauvegarde du Modele <p>
Etape 6: Chargement du Modele <p>
Etape 7: Predicition de l'Image <p>
Etape 8: Affichage du Résultat

## Etape 0: Libération du cache de la carte graphique

In [2]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()
    
free_gpu_cache()

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  8% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  3% |  9% |


## Etape 1: Chargement des Dataset

In [3]:
data_dir = 'car_data/'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

## Etape 2: Transformation des Datasets

In [4]:
#construction du modele du dataset d'entrainement
train_transforms = transforms.Compose([transforms.Resize((244,244)),
                                       transforms.RandomRotation(30),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


#construction du modele du dataset de test
test_transforms = transforms.Compose([transforms.Resize((244,244)),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#construction du modele du dataset de alidation
validation_transforms = transforms.Compose([transforms.Resize((244,244)),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


# Chargement des datasets avec la methode ImageFolder
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)
valid_data = datasets.ImageFolder(data_dir + '/valid', transform=validation_transforms)


# Utilisation des Datasets d'images et création des models de transformation
# Shuffle est a "True", ce qui signifie que l'odre des images n'affecte pas la création des modeles
trainloader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=32, shuffle=True)

## Etape 3: Creation du Modele

In [5]:
#model = models.densenet121(pretrained=True)
model = models.resnet34(pretrained=True)

In [6]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 196)


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)

## Etape 4: Entrainement du Modele

In [8]:
# Fonction de validation du Modele
def validation(model, validloader, criterion):
    valid_loss = 0
    accuracy = 0
    
    # change model to work with cuda
    model.to('cuda')

    # Iterate over data from validloader
    for ii, (images, labels) in enumerate(validloader):
    
        # Change images and labels to work with cuda
        images, labels = images.to('cuda'), labels.to('cuda')

        # Forward pass image though model for prediction
        output = model.forward(images)
        # Calculate loss
        valid_loss += criterion(output, labels).item()
        # Calculate probability
        ps = torch.exp(output)
        
        # Calculate accuracy
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return valid_loss, accuracy

In [9]:
epochs = 10 # Nombre de passage à travers le datasets
steps = 0
print_every = 40

# Passage au mode GPU
model.to('cuda')
model.train()


for e in range(epochs):

    running_loss = 0
    
    #Iteration à travers les données pour effectuer les étapes d'entrainement
    for ii, (inputs, labels) in enumerate(trainloader):
        steps += 1
        
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        
        # Mise à zero des paramètres gradiants
        optimizer.zero_grad()
        
        # Forward >> Calcul de la prédiction pour la couche suivante
        # Backward >> Calcul du gradient pour la couche précédente
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # Réalisation des étapes de validations
        if steps % print_every == 0:
            
            # Paramétrage du mode evaluation pendant la validation
            model.eval()
            
            # Les gradients sont mis à zeros lorsqu'ils ne sont plus en entraienements
            with torch.no_grad():
                valid_loss, accuracy = validation(model, validloader, criterion)
            
            print(f"No. epochs: {e+1}, \
            Training Loss: {round(running_loss/print_every,3)} \
            Valid Loss: {round(valid_loss/len(validloader),3)} \
            Valid Accuracy: {round(float(accuracy/len(validloader)),3)}")
            
            # Retour à l'entrainement du modele
            model.train()
            lrscheduler.step(accuracy * 100)
           

RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 6.00 GiB total capacity; 4.76 GiB already allocated; 0 bytes free; 4.91 GiB reserved in total by PyTorch)
Exception raised from malloc at ..\c10\cuda\CUDACachingAllocator.cpp:272 (most recent call first):
00007FF9F1C975A200007FF9F1C97540 c10.dll!c10::Error::Error [<unknown file> @ <unknown line number>]
00007FF9E9D59BB600007FF9E9D59B40 c10_cuda.dll!c10::CUDAOutOfMemoryError::CUDAOutOfMemoryError [<unknown file> @ <unknown line number>]
00007FF9E9D6064600007FF9E9D5F320 c10_cuda.dll!c10::cuda::CUDACachingAllocator::init [<unknown file> @ <unknown line number>]
00007FF9E9D607EA00007FF9E9D5F320 c10_cuda.dll!c10::cuda::CUDACachingAllocator::init [<unknown file> @ <unknown line number>]
00007FF9E9D5504900007FF9E9D54E60 c10_cuda.dll!c10::cuda::CUDAStream::unpack [<unknown file> @ <unknown line number>]
00007FF980231FE100007FF980231EA0 torch_cuda.dll!at::native::empty_cuda [<unknown file> @ <unknown line number>]
00007FF9803

In [ ]:
correct = 0
total = 0
model.to('cuda')

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to('cuda'), labels.to('cuda')
        # Get probabilities
        outputs = model(images)
        # Turn probabilities into predictions
        _, predicted_outcome = torch.max(outputs.data, 1)
        # Total number of images
        total += labels.size(0)
        # Count number of cases in which predictions are correct
        correct += (predicted_outcome == labels).sum().item()

print(f"Test de Précision du Modele: {round(100 * correct / total,3)}%")

##  Etape 5: Sauvegarde du Modele

In [ ]:
# Sauvegarde des paramètres du modèle
checkpoint = {'state_dict': model.state_dict(),
              'model': model.fc,
              'class_to_idx': train_data.class_to_idx,
              'opt_state': optimizer.state_dict,
              'num_epochs': epochs}

torch.save(checkpoint, data_dir + 'sauvegarde.pth')

## Etape 6: Chargement du Modele

In [ ]:
# Fonction qui charge les données de sauvegarde et reconstruit le modele
def load_checkpoint(filepath):

    checkpoint = torch.load(filepath)
    
    #model.load_state_dict(checkpoint['state_dict'])
    model.load_state_dict(checkpoint['state_dict'], strict=False)
    model.class_to_idx = checkpoint['class_to_idx']
    
    return model

In [ ]:
# Chargement du modele
model = load_checkpoint(data_dir + 'sauvegarde.pth')
# Vérification du nombre de sorties du classisficateur
print(model)


In [ ]:
model = torch.nn.DataParallel(model)

## Etape 7: Predicition de l'Image

In [ ]:
def process_image(image):
    

    # Traitemement d'une image avec le modele Pytorch
    
    # Convertion de l'image en format PIL image 
    pil_im = Image.open(f'{image}' + '.jpg')

    # Cosntruction de la trasnformation d'image
    transform = transforms.Compose([transforms.Resize((244,244)),
                                    #transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], 
                                                         [0.229, 0.224, 0.225])]) 
    
    # Trasnformation de l'image pour l'utiliser dans le réseau neural
    pil_tfd = transform(pil_im)
    
    # Convertion en tableau Numpy
    array_im_tfd = np.array(pil_tfd)
    
    return array_im_tfd

In [ ]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors suppose que le canal de couleur est la première dimension
    # Cependant matplotlib suppose que c'est la troisième dimension
    image = image.transpose((1, 2, 0))
    
    # Annulation du prétraitement
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # L'Image à besoin d'etre coupé entre 0 et 1 pour etre correctement affiché 
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
imshow(process_image(data_dir + 'bmw 3 2019'))

In [ ]:
def predict(image_path, model, topk=5): 
    # Implémentation du code pour prédire une classe à aprtir d'une image
    
    # Chargement du modele
    loaded_model = load_checkpoint(model).cpu()
    # Pré traitementdu modele
    img = process_image(image_path)
    # Convertion du torch tensor en image
    img_tensor = torch.from_numpy(img).type(torch.FloatTensor)
    # Adding dimension to image to comply with (B x C x W x H) input of model
    img_add_dim = img_tensor.unsqueeze_(0)

    # Paramétrage du model du mode évaluation et mise à zero des gradients
    loaded_model.eval()
    with torch.no_grad():
        # Exécution de l'image à travers du réseau neural
        output = loaded_model.forward(img_add_dim)
        
    #conf, predicted = torch.max(output.data, 1)   
    probs_top = output.topk(topk)[0]
    predicted_top = output.topk(topk)[1]
    
    # Convertion des probalités et des résultats en listes
    conf = np.array(probs_top)[0]
    predicted = np.array(predicted_top)[0]
        
    #return probs_top_list, index_top_list
    return conf, predicted

In [ ]:
# Rattache les indices de classes à leur noms
def find_classes(dir):
    classes = os.listdir(dir)
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx
classes, c_to_idx = find_classes(data_dir+"train")

print(classes, c_to_idx)

## Etape 8: Affichage du Résultat

In [ ]:
model_path = '/content/drive/My Drive/Colab Notebooks/my_checkpoint1.pth'
image_path = data_dir + 'bmw 3 2010'


conf1, predicted1 = predict(image_path, model_path, topk=5)

print(conf1)
print(classes[predicted1[0]])


In [ ]:
# Test de la fonction de prédiction

# Les Inputs sont les chemins du modele suavegardé et de l'image
model_path =  data_dir + 'sauvegarde.pth'
carname = 'Hyundai Veloster Hatchback 2012'
image_path = data_dir + carname


conf2, predicted1 = predict(image_path, model_path, topk=5)
# Convertion des classes en noms
names = []
for i in range(5):
  
    names += [classes[predicted1[i]]]

# Creation de limage PIL
image = Image.open(image_path+'.jpg')

#Affichage de l'image test et des probabilités
f, ax = plt.subplots(2,figsize = (6,10))

ax[0].imshow(image)
ax[0].set_title(carname)

y_names = np.arange(len(names))
ax[1].barh(y_names, conf2/conf2.sum(), color='darkblue')
ax[1].set_yticks(y_names)
ax[1].set_yticklabels(names)
ax[1].invert_yaxis() 

plt.show()

In [ ]:
def plot_solution(cardir, model):
 # Test de la fonction de prédiction

  # Les Inputs sont les chemins du modele suavegardé et de l'image
  model_path = data_dir + 'sauvegarde.pth'
  image_path = test_dir + cardir
  carname = cardir.split('/')[1]

  conf2, predicted1 = predict(image_path, model_path, topk=5)
  # Convertion des classes en noms
  names = []
  for i in range(5):
  
      names += [classes[predicted1[i]]]


  # Creation de limage PIL
  image = Image.open(image_path+'.jpg')

  #Affichage de l'image test et des probabilités
  f, ax = plt.subplots(2,figsize = (6,10))

  ax[0].imshow(image)
  ax[0].set_title(carname)

  y_names = np.arange(len(names))
  ax[1].barh(y_names, conf2/conf2.sum(), color='darkblue')
  ax[1].set_yticks(y_names)
  ax[1].set_yticklabels(names)
  ax[1].invert_yaxis() 

  plt.show()

In [ ]:
cardir='/Mercedes-Benz S-Class Sedan 2012/06543'
plot_solution(cardir, model)

In [ ]:
cardir='/BMW 3 Series Sedan 2012/06582'
plot_solution(cardir, model)

In [ ]:
cardir='/BMW 3 Series Sedan 2012/06544'
plot_solution(cardir, model)

In [ ]:
cardir='/BMW M5 Sedan 2010/03529'
plot_solution(cardir, model)

In [ ]:
cardir='/BMW X6 SUV 2012/02891'
plot_solution(cardir, model)

In [ ]:
cardir='/BMW X5 SUV 2007/03310'
plot_solution(cardir, model)

In [ ]:
cardir='/Hyundai Veloster Hatchback 2012/06652'
plot_solution(cardir, model)

In [ ]:
cardir='/Volkswagen Golf Hatchback 2012/06875'
plot_solution(cardir, model)

In [ ]:
cardir='/Hyundai Tucson SUV 2012/07220'
plot_solution(cardir, model)

In [ ]:
imageFileName = input("Entrer l'URL de l'image la voiture")
plot_solution(imageFileName, model)